# TREC 2018 Precision Medicine

In [1]:
import json
from json2html import *
from IPython.display import HTML
import pandas

import os, sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from trec_utils import utils, running, evaluation

In [2]:
config = utils.load_config()

In [3]:
topics_all = utils.get_topics('./topics/topics2017.xml')
qrels_all = utils.get_qrels('./gold-standard/trials.2017.qrels')
#topics_all

# Sample investigation of missing documents in a topic

In [4]:
TOPIC = 1
topic = topics_all[(topics_all.topic==TOPIC)]
qrels = utils.qrels_of_topics(qrels_all, topic)
topic

,topic,disease,gene,gene1,gene2,gene3,sex,age,age_group
1,1,Liposarcoma,CDK4 Amplification,CDK4 Amplification,,,male,38,adult


In [5]:
run_params = {
    'run_id':'imi_mug_',
    'query_template':'submitted/imi_mug_ct1.json',
}

run, run_params = running.run(topic, 'TRIALS', run_params)
results, aggregated = evaluation.evaluate(qrels[(qrels.relev.isin([2]))], run)
aggregated

RUN: imi_mug_ TOPICS: 1


{'P_10': 0.4, 'Rprec': 0.8, 'ndcg': 0.786, 'recall_1000': 0.8}

Check relevant documents missed and why (for topic 2 for example)

In [6]:
run_set = set(run[(run.TOPIC_NO==TOPIC)].ID)
qrels_set = set(qrels[(qrels.relev.isin([2]))].doc_id)
qrels_set.difference(run_set)

{'NCT02846987'}

## Best Runs

In [7]:
run_params = {
    'run_id':'DEFAULT_RUN',
    'query_template':'tests/baseline-ct.json',
    'disease_tie_breaker':0.4,
    'disease_multi_match_type':'best_fields',
    'disease_boost':1.5,
    'gene_tie_breaker':0.5,
    'gene_multi_match_type':'cross_fields',
    'gene_boost':1
}

run, params = running.run(topics_all, 'TRIALS', run_params)
results, aggregated = evaluation.evaluate(qrels_all, run)

aggregated

RUN: DEFAULT_RUN TOPICS: 30


{'P_10': 0.3345, 'Rprec': 0.2554, 'ndcg': 0.389, 'recall_1000': 0.4402}

In [8]:
run_params = {
    'run_id':'DEFAULT_RUN',
    'query_template':'variable/shingles.json',
    'disease_tie_breaker':0.4,
    'disease_multi_match_type':'best_fields',
    'disease_boost':1.3,
    'gene_tie_breaker':0.4,
    'gene_multi_match_type':'best_fields',
    'gene_boost':0.8
}

run, params = running.run(topics_all, 'TRIALS', run_params)
results, aggregated = evaluation.evaluate(qrels_all, run)

aggregated

RUN: DEFAULT_RUN TOPICS: 30


{'P_10': 0.3241, 'Rprec': 0.2523, 'ndcg': 0.3948, 'recall_1000': 0.4402}

## Experiment and do grid search

In [21]:
# Phrase and phrase prefix give really bad results in the multi match type, discard

experiment_params_grid = {
    'query_template':['variable/shingles.json'],
    'disease_tie_breaker':[0.4,0.6],
    'disease_multi_match_type':['best_fields', 'cross_fields'],
    'disease_boost':[1.3,1.6],
    'gene_tie_breaker':[0.4,0.6],
    'gene_multi_match_type':['best_fields', 'cross_fields'],
    'gene_boost':[0.8,1.1]
}

results = running.experiment(topics_all, qrels_all, 'TRIALS', experiment_params_grid)

EXPERIMENT BEGIN: 2018-08-02 15:35:39.394637
RUNS: 64
1
RUN: variable/shingles.json-0.4-best_fields-1.3-0.4-best_fields-0.8 TOPICS: 30
('variable/shingles.json', 0.3959, 0.3172, 0.2528, '0.4', 'best_fields', '1.3', '0.4', 'best_fields', '0.8')
2
RUN: variable/shingles.json-0.4-best_fields-1.3-0.4-best_fields-1.1 TOPICS: 30
('variable/shingles.json', 0.3959, 0.3172, 0.2528, '0.4', 'best_fields', '1.3', '0.4', 'best_fields', '1.1')
3
RUN: variable/shingles.json-0.4-best_fields-1.3-0.4-cross_fields-0.8 TOPICS: 30
('variable/shingles.json', 0.3959, 0.3172, 0.2528, '0.4', 'best_fields', '1.3', '0.4', 'cross_fields', '0.8')
4
RUN: variable/shingles.json-0.4-best_fields-1.3-0.4-cross_fields-1.1 TOPICS: 30
('variable/shingles.json', 0.3959, 0.3172, 0.2528, '0.4', 'best_fields', '1.3', '0.4', 'cross_fields', '1.1')
5
RUN: variable/shingles.json-0.4-best_fields-1.3-0.6-best_fields-0.8 TOPICS: 30
('variable/shingles.json', 0.3959, 0.3172, 0.2528, '0.4', 'best_fields', '1.3', '0.6', 'best_fields',

KeyboardInterrupt: 

In [ ]:
results

## FIXME: Load and split TOPICS and GOLD STANDARD from last year (30 topics)
Also, split them into training, test, and development set.

In [4]:
topics_train, topics_test, topics_dev = utils.split_topics(topics_all)
qrels_train, qrels_test, qrels_dev = utils.split_qrels(qrels_all, topics_train, topics_test, topics_dev)

In [5]:
assert(set([topic for topic in topics_train['topic']])==set([topic for topic in qrels_train['topic']]))
assert(set([topic for topic in topics_test['topic']])==set([topic for topic in qrels_test['topic']]))
assert(set([topic for topic in topics_dev['topic']])==set([topic for topic in qrels_dev['topic']]))

### Example runs

In [6]:
full_run_params = running.get_default_run_params()
full_run_params['query_template'] = 'baseline.json'
full_run_df, full_run_params = running.run(topics_all)
results, aggregated = evaluation.evaluate(qrels_all, full_run_df)
aggregated
#assert(aggregated['ndcg'] == 0.5338)
#{'P_10': 0.48, 'Rprec': 0.2847, 'ndcg': 0.5338, 'recall_1000': 0.6139}

RUN: DEFAULT_RUN TOPICS: 30 {'run_id': 'DEFAULT_RUN', 'gene_tie_breaker': 0.5, 'gene_boost': 1, 'query_template': 'baseline.json', 'disease_tie_breaker': 0.5, 'disease_boost': 1.5}


{'P_10': 0.48, 'Rprec': 0.2847, 'ndcg': 0.5338, 'recall_1000': 0.6139}

In [6]:
run_params = running.get_default_run_params()
run_params['query_template'] = 'fixed/baseline_sex_age.json'
full_run_df, run_params = running.run(topics_all)
results, aggregated = evaluation.evaluate(qrels_all, full_run_df)
aggregated
# Only MeSH, best fields: {'P_10': 0.5167, 'Rprec': 0.2785, 'ndcg': 0.5383, 'recall_1000': 0.6148}
# Only MeSH, cross fields: {'P_10': 0.5067, 'Rprec': 0.2811, 'ndcg': 0.5413, 'recall_1000': 0.6159}
# Only MeSH, cross, best: {'P_10': 0.52, 'Rprec': 0.2814, 'ndcg': 0.5378, 'recall_1000': 0.6151}
# Only MeSH, best, cross: {'P_10': 0.52, 'Rprec': 0.2787, 'ndcg': 0.5417, 'recall_1000': 0.6173} ***** best ndcg, best P_10

RUN: DEFAULT_RUN TOPICS: 30


{'P_10': 0.52, 'Rprec': 0.2787, 'ndcg': 0.5417, 'recall_1000': 0.6173}

In [7]:
run_params = running.get_default_run_params()
run_params['query_template'] = 'fixed/baseline_sex_age_all_fields.json'
full_run_df, run_params = running.run(topics_all)
results, aggregated = evaluation.evaluate(qrels_all, full_run_df)
aggregated
# All fields, best fields: {'P_10': 0.51, 'Rprec': 0.2744, 'ndcg': 0.5396, 'recall_1000': 0.6143}
# All fields, cross fields: {'P_10': 0.5067, 'Rprec': 0.281, 'ndcg': 0.5412, 'recall_1000': 0.6157}
# All fields, best fields disease, gene, cross fields rest: {'P_10': 0.5167, 'Rprec': 0.276, 'ndcg': 0.5378, 'recall_1000': 0.6139}
# All fields, best fields disease, cross fields rest: {'P_10': 0.52, 'Rprec': 0.278, 'ndcg': 0.5416, 'recall_1000': 0.6174} ***** best ndcg, best P_10

RUN: DEFAULT_RUN TOPICS: 30


{'P_10': 0.52, 'Rprec': 0.278, 'ndcg': 0.5416, 'recall_1000': 0.6174}

In [6]:
#full_run_df

Test run with default_params

In [6]:
training_run_df, training_run_params = running.run(topics_train)

RUN: DEFAULT_RUN TOPICS: 12 {'run_id': 'DEFAULT_RUN', 'disease_tie_breaker': 0.5, 'gene_boost': 1, 'gene_tie_breaker': 0.5, 'query_template': 'baseline.json', 'disease_boost': 1.5}


In [7]:
training_results, training_aggregated = evaluation.evaluate(qrels_train, training_run_df)
training_score = pandas.Series(training_aggregated, name='training ' + '(' + str(len(topics_train)) + ' topics)')

In [8]:
test_run_df, test_run_params = running.run(topics_test, training_run_params)

RUN: DEFAULT_RUN TOPICS: 9 {'run_id': 'DEFAULT_RUN', 'disease_tie_breaker': 0.5, 'gene_boost': 1, 'gene_tie_breaker': 0.5, 'query_template': 'baseline.json', 'disease_boost': 1.5}


In [9]:
training_score

P_10           0.4333
Rprec          0.2810
ndcg           0.5383
recall_1000    0.6650
Name: training (12 topics), dtype: float64

In [10]:
test_results, test_aggregated = evaluation.evaluate(qrels_test, test_run_df)
test_score = pandas.Series(test_aggregated, name='test ' + '(' + str(len(topics_test)) + ' topics)')

In [11]:
pandas.DataFrame([training_score, test_score])

,P_10,Rprec,ndcg,recall_1000
training (12 topics),0.4333,0.2810,0.5383,0.6650
test (9 topics),0.3556,0.2194,0.4820,0.5423
